In [11]:
from os_helper import OsHelper
os_helper = OsHelper()

In [12]:
from compute_metrics import ComputeMetrics

price_data = os_helper.read_data(directory_name="base data", file_name="close_fixed.csv", index_col=0, low_memory=False)
trade_volume = os_helper.read_data(directory_name="base data", file_name="volume_fixed.csv", index_col=0, low_memory=False)
spy = os_helper.read_data(directory_name="benchmark", file_name="SPY (1).csv", index_col=0, low_memory=False)

print(price_data.head())
print(trade_volume.head())
print(spy.head())

rolling_window_years = [1, 2, 3]

# Compute all risk metrics needed for the strategy and the filter
metrics_calculator = ComputeMetrics(price_data=price_data, trade_volume=trade_volume, rolling_window_years=rolling_window_years,
                                    benchmark_series=spy['Adj Close'])

final_df = metrics_calculator.compile_all_metrics()

os_helper.write_data(directory_name="transform data", file_name="all_metrics.csv", data_frame=final_df)

Loaded file close_fixed.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file volume_fixed.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file SPY (1).csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/benchmark.
            CSCO US Equity  ISRG US Equity  TPR US Equity  DVN US Equity  MRO US Equity  BA US Equity  GILD US Equity  MDT US Equity  V US Equity  A US Equity  MO US Equity  CTRA US Equity  MCHP US Equity  EIX US Equity  BBY US Equity  GEN US Equity  WBA US Equity  DTE US Equity  C US Equity  T US Equity  ...  RHI US Equity  EFX US Equity  NUE US Equity  CTSH US Equity  SWK US Equity  MU US Equity  TRV US Equity  L US Equity  AEP US Equity  CI US Equity  BALL US Equity  JNJ US Equity  DOV US Equity  CRM US Equity  PGR US Equity  WAT US Equity  BWA US Equity  BLK US Equity  PPL

In [13]:
from get_rebal_dates import GetRebalDates, filter_by_rebalance_dates

all_metrics = os_helper.read_data(directory_name="transform data", file_name="all_metrics.csv", index_col=0, header=[0, 1])
print(all_metrics.head())

termination_date = '2014-12-31' # Last day of backtest
initial_year = 2000 # First year of the backtest
reb_month = 1
reb_week = 2
reb_weekday = 'FRI'
reb_frequency = 'S' # Q for quarterly, M for monthly, A for annual, S for semi-annual

# Get the rebalance dates of the strategy
get_reb_dates = GetRebalDates(termination_date=termination_date, initial_year=initial_year, reb_month=reb_month,
                              reb_week=reb_week, reb_weekday=reb_weekday, reb_frequency=reb_frequency)
rebalance_dates = get_reb_dates.reb_dates

print("Dates de rebalancement :")
for date in rebalance_dates:
    print(date)

reb_metrics = filter_by_rebalance_dates(get_reb_dates=get_reb_dates, data=all_metrics)
print(reb_metrics.head())

os_helper.write_data(directory_name="transform data", file_name=f"rebalance_metrics.csv", data_frame=reb_metrics)

Loaded file all_metrics.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/transform data.
                   A US Equity                                                                                                                AAPL US Equity                                                                                                                 ABT US Equity                      ... YUM US Equity                     ZBH US Equity                                                                                                                ZION US Equity                                                                                                          
           average_turnover_1Y average_turnover_2Y average_turnover_3Y beta_1Y beta_2Y beta_3Y volatility_1Y volatility_2Y volatility_3Y average_turnover_1Y average_turnover_2Y average_turnover_3Y beta_1Y beta_2Y beta_3Y volatility_1Y volatility_2Y volatility_3Y aver

In [14]:
from filter import Filter

data = os_helper.read_data(directory_name="transform data", file_name="rebalance_metrics.csv", index_col=0)
print(data)
risk_filter = Filter(data=data)

risk_top_n = 140
liquidity_top_n = 200

filtered_data = risk_filter.apply_filters(liquidity_top_n=liquidity_top_n, risk_top_n=risk_top_n)
print(filtered_data.head())

os_helper.write_data(directory_name="transform data", file_name="filtered_data.csv", data_frame=filtered_data)

Loaded file rebalance_metrics.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/transform data.
            Date          Ticker  average_turnover_1Y  average_turnover_2Y  average_turnover_3Y   beta_1Y   beta_2Y   beta_3Y  volatility_1Y  volatility_2Y  volatility_3Y
0     2000-01-07  AAPL US Equity         2.822167e+08         2.080641e+08         1.553083e+08  1.351092  1.165324  0.983969       0.602578       0.574981       0.618927
1     2000-01-07   ABT US Equity         6.775778e+07         5.411452e+07         4.707382e+07  0.932067  0.841249  0.866634       0.361892       0.323613       0.305512
2     2000-01-07  ADBE US Equity         8.940289e+07         6.678857e+07         6.509780e+07  0.966219  0.927739  0.872973       0.496930       0.521665       0.515052
3     2000-01-07   ADI US Equity         6.310535e+07         4.873519e+07         4.258878e+07  1.244670  1.140282  1.220119       0.604444       0.623926       0.6

In [15]:
from weighting import InverseMetricsWeighting

filtered_data = os_helper.read_data(directory_name="transform data", file_name="filtered_data.csv", index_col=0)
print(filtered_data)

years_to_inverse = [1, 2, 3]
metric_to_inverse = "beta"
weighting_list = [] # Equal weighting of betas
min_weight = 0.00001
max_weight = 0.05

inverse_metrics_weighting = InverseMetricsWeighting(filtered_data=filtered_data, years_list=years_to_inverse,
                                                    target_metric=metric_to_inverse, weight_list=weighting_list,
                                                    min_limit=min_weight, max_limit=max_weight)
inverse_metrics_weighting_df = inverse_metrics_weighting.compute_inverse_weighted_metrics()
print(inverse_metrics_weighting_df)

os_helper.write_data(directory_name="transform data", file_name="inverse_metrics_weighting.csv", data_frame=inverse_metrics_weighting_df)

Loaded file filtered_data.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/transform data.
            Date          Ticker  average_turnover_1Y  average_turnover_2Y  average_turnover_3Y   beta_1Y   beta_2Y   beta_3Y  volatility_1Y  volatility_2Y  volatility_3Y
0     2000-01-07   ADM US Equity         1.685440e+07         1.982424e+07         2.319849e+07  0.376382  0.464531  0.516650       0.321483       0.331964       0.319116
1     2000-01-07   ADP US Equity         4.139615e+07         3.827311e+07         3.374083e+07  0.744642  0.792819  0.766803       0.296544       0.293345       0.284797
2     2000-01-07   AEP US Equity         1.684560e+07         1.810501e+07         1.777850e+07  0.275702  0.248073  0.278439       0.209497       0.197221       0.180720
3     2000-01-07   AES US Equity         4.062485e+07         3.431749e+07         2.833713e+07  0.460641  0.691590  0.636934       0.396610       0.464803       0.43924

In [16]:
from portfolio_returns import PortfolioReturns

prices = os_helper.read_data(directory_name="base data", file_name="tot_ret_fixed.csv", index_col=0, sep=',')
print(prices.head())
weighting_df = os_helper.read_data(directory_name="transform data", file_name="inverse_metrics_weighting.csv", index_col=0)
print(weighting_df.head())

rebalance_lag = 7 # delay between weight computation and implementation
transaction_fee = 0.001 # 0.1% transaction fee

pf_returns = PortfolioReturns(prices=prices, weighting_data=weighting_df, rebalance_lag=rebalance_lag, transaction_fee=transaction_fee)
drifted_weights = pf_returns.drifted_weights
print(drifted_weights.head())
portfolio_returns = pf_returns.portfolio_returns
print(portfolio_returns.head())

os_helper.write_data(directory_name="final data", file_name="portfolio_returns.csv", data_frame=portfolio_returns)
os_helper.write_data(directory_name="final data", file_name="drifted_weights.csv", data_frame=drifted_weights)

Loaded file tot_ret_fixed.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
            CSCO US Equity  ISRG US Equity  TPR US Equity  DVN US Equity  MRO US Equity  BA US Equity  GILD US Equity  MDT US Equity  V US Equity  A US Equity  MO US Equity  CTRA US Equity  MCHP US Equity  EIX US Equity  BBY US Equity  GEN US Equity  WBA US Equity  DTE US Equity  C US Equity  T US Equity  ...  WYNN US Equity  RHI US Equity  EFX US Equity  NUE US Equity  CTSH US Equity  SWK US Equity  MU US Equity  TRV US Equity  L US Equity  AEP US Equity  CI US Equity  BALL US Equity  JNJ US Equity  DOV US Equity  CRM US Equity  PGR US Equity  WAT US Equity  BWA US Equity  BLK US Equity  PPL US Equity
Date                                                                                                                                                                                                                                                    

In [17]:
from fama_regress import FamaFrenchRegression

ff_factors = os_helper.read_data(directory_name='benchmark', file_name='F-F_Research_Data_5_Factors_2x3_daily.csv')
portfolio_returns = os_helper.read_data(directory_name='final data', file_name='portfolio_returns.csv')

print(ff_factors.head())
print(portfolio_returns.head())

# Instantiate the FamaFrenchRegression class
ff_regression = FamaFrenchRegression(ff_factors, portfolio_returns)

# Run the regression
ff_regression.run_regression()

# Retrieve and display the regression results
regression_results = ff_regression.get_regression_results()
print(regression_results)

 # Run the rolling regression
rolling_results = ff_regression.run_rolling_regression(window_size=252)

# Print or use the rolling regression results
print(rolling_results)


Loaded file F-F_Research_Data_5_Factors_2x3_daily.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/benchmark.
Loaded file portfolio_returns.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/final data.
   Unnamed: 0  Mkt-RF   SMB   HML   RMW   CMA     RF
0    19630701   -0.67  0.02 -0.35  0.03  0.13  0.012
1    19630702    0.79 -0.28  0.28 -0.08 -0.21  0.012
2    19630703    0.63 -0.18 -0.10  0.13 -0.25  0.012
3    19630705    0.40  0.09 -0.28  0.07 -0.30  0.012
4    19630708   -0.63  0.07 -0.20 -0.27  0.06  0.012
   Unnamed: 0  Portfolio_Returns
0  2000-01-14           0.000000
1  2000-01-18           0.007622
2  2000-01-19          -0.001033
3  2000-01-20          -0.006445
4  2000-01-21          -0.009305
                                 OLS Regression Results                                
Dep. Variable:          Excess_Return   R-squared (uncentered):                   0

In [18]:
from perf_report import PerfReport

benchmark_prices = os_helper.read_data(directory_name="benchmark", file_name="SPY (1).csv", index_col=0)
benchmark_prices.sort_index(inplace=True)
benchmark_prices.index.name = None
benchmark_prices = benchmark_prices[["Adj Close"]]
benchmark_prices.columns = ["SPY"]

print(benchmark_prices.head())

strategy_returns = os_helper.read_data(directory_name="final data", file_name="portfolio_returns.csv", index_col=0)
print(strategy_returns.head())
strategy_returns.columns = ["Portfolio_Returns"]

perf_report = PerfReport(port_returns=strategy_returns, bench_returns=benchmark_prices, strat_name="InverseMetrics")
print(perf_report.get_performance_indicators(rf=0.01, periods_per_year=252))
perf_report.generate_html_report(rf=0.01, periods_per_year=252)

Loaded file SPY (1).csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/benchmark.
                  SPY
1995-01-03  27.352005
1995-01-04  27.482687
1995-01-05  27.482687
1995-01-06  27.510689
1995-01-09  27.538696
Loaded file portfolio_returns.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/final data.
            Portfolio_Returns
2000-01-14           0.000000
2000-01-18           0.007622
2000-01-19          -0.001033
2000-01-20          -0.006445
2000-01-21          -0.009305
{'Cumulative Returns': 8.626309625980245, 'CAGR': 0.15231620683851443, 'Sharpe': 1.652652410736296, 'Sortino': 2.4436995649045157, 'Calmar': 1.6734970349078595, 'Max Drawdown': -0.09101671748519158, 'Average Return': 0.0005775231060992804, 'Volatility': 0.0820190958210068, 'Tail Ratio': 1.073163657650231, 'Gain to Pain Ratio': 0.35138239262976195, 'Value at Risk (VaR)': -0.00792111453472652, 'Condition

/Users/thomasvaudescal/.virtualenvs/PolyHacks/lib/python3.11/site-packages/quantstats/stats.py:968: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  returns = returns.pivot('Year', 'Month', 'Returns').fillna(0)


Moved file InverseMetrics_backtesting_report.html to folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Codes/reports.


In [19]:
from stickyRebalance import prepare_drifted_weights_format_submission

drifted_weights = os_helper.read_data(directory_name='final data', file_name='drifted_weights.csv', index_col=0)
print(drifted_weights.head())

submission_weights = prepare_drifted_weights_format_submission(drifted_weights=drifted_weights)

print(submission_weights.head())

os_helper.write_data(directory_name='final data', file_name='submission_weights.csv', data_frame=submission_weights)

Loaded file drifted_weights.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/final data.
            A US Equity  ABT US Equity  ACN US Equity  ADI US Equity  ADM US Equity  ADP US Equity  AEE US Equity  AEP US Equity  AES US Equity  AFL US Equity  AIZ US Equity  AKAM US Equity  ALL US Equity  AMP US Equity  AMT US Equity  AON US Equity  APA US Equity  APD US Equity  APH US Equity  AVB US Equity  ...  TXT US Equity  UNH US Equity  UNP US Equity  VFC US Equity  VLO US Equity  VMC US Equity  VRSN US Equity  VTR US Equity  VTRS US Equity  WAT US Equity  WBA US Equity  WEC US Equity  WELL US Equity  WMB US Equity  WYNN US Equity  XEL US Equity  XRAY US Equity  YUM US Equity  ZBH US Equity  ZION US Equity
2000-01-14          0.0            0.0            0.0            0.0       0.003869       0.003701            0.0       0.004188       0.003772            0.0            0.0             0.0            0.0            0.0            0.0